# Lab 2 - Constructing the Graph Database
In this lab we will ingest the product metadata into Neptune and construct a graph from it.

## What is a Graph Database?

A graph database is a database that stores relationships with a graph structure. Data is represented by nodes, edges and properties rather than tables or documents. In graph databases, we are able to transverse relationships very quickly as relationships between nodes are persisted in the database, rather than being caluclated at query times.

[Amazon Neptune](https://aws.amazon.com/neptune/) is a high-performance graph database engine optimized for storing billions of relationships and querying the graph with millisecond latency. The nature of Neptune makes it a great tool for recommendation applications, as recommendations can be made quickly based on existing relationships.

## Setup
Just as in the first lab, we have to prepare our environment by importing dependencies and creating clients.

### Update Dependencies
To get started, we need to perform a bit of setup. First, we need to ensure that a current version of gremlinpython is currently installed. [Gremlin](https://tinkerpop.apache.org/gremlin.html) is the graph traversal language of [Apache Tinkerpop](https://tinkerpop.apache.org/). TinkerPop is a popular graph computing framework for both graph databases (OLTP) and graph analytic systems (OLAP).

The following cell will update pip and install the latest gremlinpython library.

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall gremlinpython
!{sys.executable} -m pip install --upgrade --force-reinstall nest-asyncio

import nest_asyncio
nest_asyncio.apply()
__import__('IPython').embed()

### Import dependencies
The following libraries are needed for this lab

In [ ]:
import boto3
import uuid
import json
import pandas as pd
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import *

### Create Clients
Next we need to create the AWS service clients needed for this workshop.
- **neptune**: This resource is used to create our Neptune DB cluster and endpoint.

In [ ]:
neptune = boto3.client('neptune')

### Load variables saved in Lab 1
At the end of Lab 1 we saved some variables that we'll need in this lab. The following cell with load those variables into this lab environment.

In [ ]:
%store -r

## Create Neptune Cluster and DB Instancee
Currently, there is no Neptune cluster to store the relationships. We can create the Neptune DB cluster and the DB instance which will give us access to reader and a writer cluster endpoints. Creation will take a few minutes.

In [ ]:
response = neptune.create_db_subnet_group(
    DBSubnetGroupName='retail-demo-store-neptune-subnet-group',
    DBSubnetGroupDescription='Subnet group for products Neptune cluster',
    SubnetIds=[
    ],
)

neptune_cluster = neptune.create_db_cluster(
    DBClusterIdentifier='retail-demo-store-products-cluster',
    Engine='neptune',
    DBSubnetGroupName='retail-demo-store-neptune-subnet-group',
)

endpoint = neptune_cluster['DBCluster']['Endpoint']
print('Neptune Endpoint:' + endpoint)

neptune_db_instance = neptune.create_db_instance(
    DBInstanceIdentifier='retail-demo-store-products-graph',
    Engine='neptune',
    DBInstanceClass='db.r6g.large',
    DBClusterIdentifier=neptune_cluster['DBCluster']['DBClusterIdentifier']
)

#### Wait for DB Instance to have active status

In [ ]:
import time

status = None
max_time = time.time() + 15*60
while time.time() < max_time:
    response = neptune.describe_db_instances(DBInstanceIdentifier=neptune_db_instance['DBInstance']['DBInstanceIdentifier'])
    status = response['DBInstances'][0]['DBInstanceStatus']
    print("Status: {}".format(status))

    if status == 'available':
        break

    time.sleep(30)

## Data Preparation
Before we can start building the relationships between nodes and edges, we have to load the products data into a dataframe. This can be done by performing a scan on the DynamoDB table, and adding each data row to the dataframe.

In [ ]:
# DynamoDB Scan step:
dynamodb = boto3.resource('dynamodb')
ddb_response = ddb_table.scan()
items = ddb_response['Items']

# Fetch data into DF
pd_data = []
for data_row in items:
    pd_data.append(data_row)

We can drop columns from the data we are not interested in the relationships of to speed up processing.

In [ ]:
cols_to_drop = ['sk', 'url', 'aliases']
df_products = pd.DataFrame(pd_data)
df_products.drop(cols_to_drop, inplace=True, axis=1)

## Improving Access Patterns
To improve access patterns, we can create a dataframe for categories, which include a UUID for each one.

In [ ]:
df_categories = df_products[['category']].drop_duplicates(subset=['category'])
df_categories['category_id'] = [uuid.uuid4() for _ in range(len(df_categories.index))]

We do the same for product styles.

In [ ]:
df_styles = df_products[['style']].drop_duplicates(subset=['style'])
df_styles['style_id'] = [uuid.uuid4() for _ in range(len(df_styles.index))]

## Ingest into Neptune
As Neptune is a graph database, we have to insert the data according to this structure. This involves initializing a graph and traversing to add each product item by item.

In [ ]:
# Initialize Neptune connection with endpoint from created cluster

graph = Graph()

port = 8182
endpoint = neptune_cluster['DBCluster']['Endpoint']
endpoint = f'wss://{endpoint}:{port}/gremlin'

graph=Graph()

connection = DriverRemoteConnection(endpoint,'g',
                 transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))
g = graph.traversal().withRemote(connection)

In [ ]:
# Insert all products
for index, row in df_products.iterrows():

    # Insert item by item.
    vertex_insert = g.addV('product') \
        .property(T.id, row['id']) \
        .property('product_name', row['name']) \
        .property('style', row['style']) \
        .property('gender_affinity', row['gender_affinity']) \
        .property('category', row['category']) \
        .property('featured', row['featured']) \
        .next()

# Need performance improvements and optimizations
# Updating items, to add Labels array (Neptune does not support dict/maps in addV).
    for prop_label in json.loads(row['image_labels']):
        if prop_label['Confidence'] > 75:
            update_results = g.V(vertex_insert).property('labels_confidence_gt_75',
                                                         prop_label['Name'].lower()).next()

We can then insert all the categories and styles created earlier as multi-label vertices to improve searchability.

In [ ]:
# Insert all categories
for index, row in df_categories.iterrows():
    g.addV('category::{}'.format(row['category'])).property(T.id, str(row['category_id'])).property(
        'name', row['category']).next()

# Insert all styles
for index, row in df_styles.iterrows():
    g.addV('style::{}'.format(row['style'])).property(T.id, str(row['style_id'])).property(
        'name', row['style']).next()

With all the insertions complete, the edges now have to be constructed to connect the graph vertices.

In [ ]:
# Add Category and Style IDs
df_with_category_ids = pd.merge(df_products, df_categories, on='category', how='inner')
df_with_cat_and_style_ids = pd.merge(df_with_category_ids, df_styles, on='style', how='inner')

# Create Edges for Categories -> Styles
df_edges_category_style = df_with_cat_and_style_ids[['category_id', 'style_id']].drop_duplicates(
    subset=['category_id', 'style_id'])
# Add edges for Categories -> Styles:
for index, row in df_edges_category_style.iterrows():
    cat_to_style_edge_insert = g.V(str(row['category_id'])).addE('has').to(__.V(str(row['style_id']))).next()
    print(cat_to_style_edge_insert)

# Create Edges for Styles --> Products
df_edges_styles_products = df_with_cat_and_style_ids[['style_id', 'id']].drop_duplicates(subset=['style_id', 'id'])
# Add edges for Styles --> Products (ID is the original column of a product_id):
for index, row in df_edges_styles_products.iterrows():
    style_to_prod_edge_insert = g.V(str(row['style_id'])).addE('has').to(__.V(str(row['id']))).next()
    print(style_to_prod_edge_insert)

### Cleanup
With all the insertions complete and edges created, we close the connection to the graph database.

In [ ]:
connection.close()

## Lab 2 Summary
In this lab we created a Neptune cluster and wrote all our products data to our instance. We then constructed graph vertices between categories and styles which showcase the relationship between products.

In the next lab, we will retrain Personalize with the image label data.

## Store Variables Needed in the Next Lab
We will pass some variables initialized in this lab by storing them in the notebook environment.

In [ ]:
db_cluster_identifier = neptune_cluster['DBCluster']['DBClusterIdentifier']
%store db_cluster_identifier


### Continue to Lab 3
Open [Lab 3](./Lab-3-Integrate-Neptune-with-OpenSearch.ipynb) to continue the workshop.